In [36]:
from dataclasses import dataclass, asdict, field 
import datetime
import pandas as pd
from typing import List

class Instrument:
    Equity="Eq"
    OptionsStock="OPTSTK"
    OptionsIndex="OPTIDX"
    FuturesStock="FUTSTK"
    FuturesIndex="FUTIDX"
class OrdStatus:
    New="N"
    Accepted="C"
    Rejected="R"
    Exited="E"

class TranType:
    Buy="B"
    Sell="S"

@dataclass
class Order:
    ord_number:str
    ordtime:datetime.datetime
    sym:str
    tsym:str
    token:str
    instrument:str
    ls:int
    avgprice:float
    qty:int
    tranType:str = TranType.Buy
    status:str = OrdStatus.New
    totalprice:float = field(init=False)

    def __post_init__(self):
        self.totalprice = self.avgprice * self.qty

    def __eq__(self, other):
        return self.token==other.token




class OrderBook:
    def __init__(self, balance) -> None:
        self.opening_balance = balance
        self._balance = balance
        self.ob_eq:List[Order]  = []
        self.ob_opt:List[Order] = []
    
    def __repr__(self) -> str:
        bal   = f'Current balance is:     {self.balance}'
        pnl   = f'Current PnL :           {self.pnl}'
        obeq  = f'Equity Order Book:\n .  {self.print_order_list(self.ob_eq)}'
        obfno = f'FnO Order Book:\n .     {self.print_order_list(self.ob_opt)}'
        return '\n'.join([bal, pnl, obeq, obfno])

    def print_order_list(self, ol):
        # li = [asdict(ord) for ord in ol]
        # df = pd.DataFrame(li)
        df = self.to_df(ol)
        if df.empty:
            return "Empty List"
        return df.__repr__()

    def to_df(self, ol):
        li = [asdict(ord) for ord in ol]
        df = pd.DataFrame(li)
        return df
    
    def to_df_eq(self):
        return self.to_df(self.ob_eq)

    def to_df_opt(self):
        return self.to_df(self.ob_opt)

    def to_csv(self, path):
        dfeq = self.to_df_eq()
        dfop = self.to_df_opt()
        df = pd.concat([dfeq, dfop], axis=0)
        df.to_csv(path, index=None)



    @property
    def balance(self):
        return self._balance

    @property
    def pnl(self):
        return self._balance - self.opening_balance
    
    def balcheck(self, ord:Order):
        if ord.tranType == TranType.Sell:
            return True
        if ord.totalprice <= self._balance:
            return True
        print('Insufficient balance!')
        return False
        
    def isFnO(self, ord:Order):
        return False if ord.instrument == Instrument.Equity else True

    def add(self, ord:Order):
        if not self.balcheck(ord):
            return False

        if self.isFnO(ord):
            return self.addFnoOrder(ord)
        else:
            return self.addEquityOrder(ord)

    def addFnoOrder(self, ord:Order):
        self.ob_opt.append(ord)
        self._balance -= ord.totalprice * (1 if ord.tranType==TranType.Buy else -1)
        return True

    def addEquityOrder(self, ord:Order):
        self.ob_eq.append(ord)
        self._balance -= ord.totalprice * (1 if ord.tranType==TranType.Buy else -1)
        return True
    
    def active_equity_qty(self, symbol:str):
        qt_active = 0
        for ord in self.ob_eq:
            if ord.sym==symbol:
                qt_active += ord.qty * (1 if ord.tranType==TranType.Buy else -1)
        return qt_active
    
    def active_fno(self, symbol):
        tsym_qty = {}
        for ord in self.ob_opt:
            if ord.sym==symbol:
                if ord.tsym in tsym_qty:
                    tsym_qty[ord.tsym] += ord.qty * (1 if ord.tranType==TranType.Buy else -1)
                else:
                    tsym_qty[ord.tsym] = ord.qty * (1 if ord.tranType==TranType.Buy else -1)

        drop_zeros=[]
        for ts, qty in tsym_qty.items():
            if qty==0:
                drop_zeros.append(ts)
        for key in drop_zeros:
            del tsym_qty[key] 

        return tsym_qty

In [126]:
import os
import pandas as pd
import datetime
# dotenv.load_dotenv()
# logger = get_logger(filename="./log")
today = datetime.datetime(2023, 7, 28)
today_str = today.strftime("%d-%m-%Y")
# api = initialize(today_str, logger)
strat_momentum_path = os.path.join("apidata/momentum/28-07-2023/report_28-07-2023.csv")
#os.makedirs(strat_momentum_path, exist_ok=True)

# ft_data = FTDataService(logger=logger, api=api, path="./apidata")

report_path = os.path.join(strat_momentum_path, f'report_{today_str}.csv')
df = pd.read_csv(strat_momentum_path, usecols=['ordtime', 'sym', 'tsym', 'token', 'instrument', 'lotsize', 'avgprice', 'qty', 'tranType'])
df['totalPrice'] = df.qty * df.avgprice
print(df.columns)
df.head()


Index(['ordtime', 'sym', 'tsym', 'token', 'instrument', 'lotsize', 'qty',
       'avgprice', 'tranType', 'totalPrice'],
      dtype='object')


,ordtime,sym,tsym,token,instrument,lotsize,qty,avgprice,tranType,totalPrice
0,2023-07-28 09:31:00,DELTACORP,DELTACORP-EQ,15044,Eq,1,493,202.50,B,99832.5
1,2023-07-28 09:31:00,RBLBANK,RBLBANK-EQ,18391,Eq,1,418,239.00,B,99902.0
2,2023-07-28 09:31:00,IEX,IEX-EQ,220,Eq,1,814,122.80,B,99959.2
3,2023-07-28 09:31:00,SUNTV,SUNTV-EQ,13404,Eq,1,183,545.80,B,99881.4
4,2023-07-28 09:31:00,BSOFT,BSOFT-EQ,6994,Eq,1,240,416.05,B,99852.0


In [127]:
gp = df.groupby(['sym', 'tsym'])

In [128]:
df_list = []
for s in df.sym.unique():
    for ts in df[df['sym']==s].tsym.unique():
        df_list.append(gp.get_group((s, ts)))

In [130]:
print(len(df_list))
for df in df_list:
    profit = 0
    for _, row in df.iterrows():
        profit += (1 if (row['tranType'] == 'S') else -1) * row['totalPrice']
    print(df)
    print(profit)

18
                ordtime        sym          tsym  token instrument  lotsize  \
0   2023-07-28 09:31:00  DELTACORP  DELTACORP-EQ  15044         Eq        1   
10  2023-07-28 09:40:00  DELTACORP  DELTACORP-EQ  15044         Eq        1   

    qty  avgprice tranType  totalPrice  
0   493    202.50        B    99832.50  
10  493    199.75        S    98476.75  
-1355.75
                ordtime      sym        tsym  token instrument  lotsize  qty  \
1   2023-07-28 09:31:00  RBLBANK  RBLBANK-EQ  18391         Eq        1  418   
11  2023-07-28 09:40:00  RBLBANK  RBLBANK-EQ  18391         Eq        1  418   

    avgprice tranType  totalPrice  
1      239.0        B     99902.0  
11     239.4        S    100069.2  
167.1999999999971
                ordtime      sym                  tsym   token instrument  \
20  2023-07-28 12:15:00  RBLBANK  RBLBANK31AUG23C237.5  172745     OPTSTK   
26  2023-07-28 12:16:00  RBLBANK  RBLBANK31AUG23C237.5  172745     OPTSTK   

    lotsize   qty  avgprice 

In [139]:
any([1, False, False])

True